## Setup

In [2]:
import pymongo # import the library
import pandas as pd

from pymongo import MongoClient

In [3]:
client = MongoClient("mongodb://localhost:27017")

In [21]:
# Read excel file
import pandas as pd
path = r'Lab5Data\Lab5.xlsx'
df_E1 = pd.read_excel(path,sheet_name='E1')
df_E2 = pd.read_excel(path,sheet_name='E2')
df_D1 = pd.read_excel(path,sheet_name='D1')
df_S2 = pd.read_excel(path,sheet_name='S2')
df_S3 = pd.read_excel(path,sheet_name='S3')
df_students = pd.read_excel(path,sheet_name='Students')
df_courses = pd.read_excel(path,sheet_name='Courses')
# Do read all the remaining sheets from the excel



# Optional : Create a unique index
# E1_collection.create_index( "sin" ,unique = True )


In [22]:
df_E1.head()

,sin,name,lot,age
0,123456789,Bob,17.4,29
1,234567890,Dave,8.5,35
2,345678901,Lisa,96.7,32
3,456789012,Bill,32.5,24
4,567890123,Jimmy,1.0,42


In [13]:
# Create a database
db = client['algebra_db']
# Create collections
E1_collection = db['E1']
E2_collection = db['E2']
D1_collection = db['D1']
S2_collection = db['S2']
S3_collection = db['S3']
Students_collection = db['Students']
Courses_collection = db['Courses']

# Drop collections to avoid duplicate key errors on insertion
E1_collection.drop()
E2_collection.drop()
D1_collection.drop()
S2_collection.drop()
S3_collection.drop()
Students_collection.drop()
Courses_collection.drop()

E1_collection.insert_many(df_E1.to_dict('records'))
E2_collection.insert_many(df_E2.to_dict('records'))
D1_collection.insert_many(df_D1.to_dict('records'))
S2_collection.insert_many(df_S2.to_dict('records'))
S3_collection.insert_many(df_S3.to_dict('records'))
Students_collection.insert_many(df_students.to_dict('records'))
Courses_collection.insert_many(df_courses.to_dict('records'))

InsertManyResult([ObjectId('67d1ade561047d1c21e4e51f'), ObjectId('67d1ade561047d1c21e4e520'), ObjectId('67d1ade561047d1c21e4e521'), ObjectId('67d1ade561047d1c21e4e522')], acknowledged=True)

In [ ]:
db.E2

Collection(Database(MongoClient(host=['cluster0-shard-00-00.lrq2u.mongodb.net:27017', 'cluster0-shard-00-01.lrq2u.mongodb.net:27017', 'cluster0-shard-00-02.lrq2u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-3rdwy9-shard-0', tls=True), 'algebra_db'), 'E2')

## 7.2 Selection (σ)
Selection corresponds to filtering rows based on a given condition in relational algebra.
Find all employees in E1 collection who is older than 33.

In [14]:
# Find employees older than 33
selection_result = db.E1.find({'age': {'$gt': 33}})

# print the result
for document in selection_result:
  print(document)


{'_id': ObjectId('67d1ade561047d1c21e4e503'), 'sin': 234567890, 'name': 'Dave', 'lot': 8.5, 'age': 35}
{'_id': ObjectId('67d1ade561047d1c21e4e506'), 'sin': 567890123, 'name': 'Jimmy', 'lot': 1.0, 'age': 42}


## 7.3 Projection (π)
Projection is used to select specific columns or fields from a relation.
Extract name and age attributes only from E1 collection.

In [15]:
# Retrieve only 'name' and 'age' fields
projection_result = db.E1.find({}, {'_id': 0, 'name': 1, 'age': 1})
for document in projection_result:
  print(document)

{'name': 'Bob', 'age': 29}
{'name': 'Dave', 'age': 35}
{'name': 'Lisa', 'age': 32}
{'name': 'Bill', 'age': 24}
{'name': 'Jimmy', 'age': 42}


## 7.4 Union (∪)
Union combines two relations.
Combine E1 and E2 collections.

In [16]:
# Union of E1_collection and E2_collection

union_result = db.E1.aggregate([
    # Obtain E1 without '_id'
    { '$project': {'_id': 0,
                   'sin': 1,
                   'name': 1,
                   'lot': 1,
                   'age': 1,
                  }
    },

    # create union with E2 without '_id'
    { '$unionWith': {'coll' : 'E2' ,
                     'pipeline' : [{'$project': {'_id': 0,
                                                 'sin': 1,
                                                 'name': 1,
                                                 'lot': 1,
                                                 'age': 1} }]
    }},

    # Group by SIN to remove duplicates
    { '$group' : {'_id': '$sin',
                  'name': {'$first': '$name'},
                  'lot': {'$first': '$lot'},
                  'age': {'$first': '$age'}
    }}
])

for document in union_result:
    print(document)


{'_id': 345678901, 'name': 'Lisa', 'lot': 96.7, 'age': 32}
{'_id': 234567890, 'name': 'Dave', 'lot': 8.5, 'age': 35}
{'_id': 567890123, 'name': 'Jimmy', 'lot': 1.0, 'age': 42}
{'_id': 678901234, 'name': 'Beth', 'lot': 91.3, 'age': 44}
{'_id': 789012345, 'name': 'Matthew', 'lot': 74.6, 'age': 26}
{'_id': 456789012, 'name': 'Bill', 'lot': 32.5, 'age': 24}
{'_id': 123456789, 'name': 'Bob', 'lot': 17.4, 'age': 29}


## 7.5 Intersection (∩)
Intersection finds the common elements between two relations.
Find the intersection of E1 and E2 collections.

In [17]:
# Intersection of E1_collection and E2_collection

intersection_result = db.E1.aggregate([
    {   # Perform the intersection using $lookup
        '$lookup':
        {
            'from': 'E2',  # The second collection
            'localField': 'sin',  # Field in E1 collection
            'foreignField': 'sin',  # Field in E2 collection
            'as': 'intersection'  # Output field to store matching records from E2
        }
    },
    {   # Only keep records where there's a match in E2
        "$match":
        {
            "intersection": {"$ne": []}
        }
    },
    {   # Flatten the matching results from E2 collection
        "$unwind": "$intersection"
    },
    {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "sin": 1,
            "name": 1,
            "lot": 1,
            "age": 1
        }
    }
])

for document in intersection_result:
    print(document)


{'sin': 234567890, 'name': 'Dave', 'lot': 8.5, 'age': 35}
{'sin': 456789012, 'name': 'Bill', 'lot': 32.5, 'age': 24}


## 7.6 Difference (−)
Difference finds the elements present in one relation but not in another.
Find the difference between E1 and E2 collections

In [18]:
# Difference of E1_collection and E2_collection

difference_result = db.E1.aggregate([
    {   # Perform the difference using $lookup
        '$lookup':
        {
            'from': 'E2',  # The second collection
            'localField': 'sin',  # Field in E1 collection
            'foreignField': 'sin',  # Field in E2 collection
            'as': 'difference'  # Output field to store matching records from E2
        }
    },
    {   # Only keep records where there's no match in E2
        "$match": {
            "difference": {"$size": 0}
        }
    },
    {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "sin": 1,
            "name": 1,
            "lot": 1,
            "age": 1
        }
    }
])

for document in difference_result:
    print(document)


{'sin': 123456789, 'name': 'Bob', 'lot': 17.4, 'age': 29}
{'sin': 345678901, 'name': 'Lisa', 'lot': 96.7, 'age': 32}
{'sin': 567890123, 'name': 'Jimmy', 'lot': 1.0, 'age': 42}


## 7.8 Join (▷◁)
A join combines rows from two relations based on a condition.
Create join between E2 and D1 collections.

In [19]:
join_result = db.E2.aggregate([

    {   # Perform the cross product using $lookup
        "$lookup": {
            "from": "D1",  # The second collection (Departments)
            "pipeline": [],  # No matching condition, we want the full cross product
            "as": "departments"  # The output field to store the result from D1
        }
    },
    {   # Flatten the resulting department data
        "$unwind": "$departments"
    },
    {   # Use $expr to compare values from different collections
        "$match": {
            "$expr": {
                "$gt": ["$lot", "$departments.budget"]  # Condition: E2.lot > D1.budget
            }
        }
    },
    {   # Optionally exclude the _id field and include relevant fields
        "$project": {
            "_id": 0,
            "sin": 1,
            "name": 1,
            "lot": 1,
            "age": 1,
            "department_name": "$departments.dname",
            "budget": "$departments.budget"
        }
    }
])

# Print the join result
for document in join_result:
    print(document)


{'sin': 678901234, 'name': 'Beth', 'lot': 91.3, 'age': 44, 'department_name': 'HR ', 'budget': 87.1}
{'sin': 678901234, 'name': 'Beth', 'lot': 91.3, 'age': 44, 'department_name': 'Sales', 'budget': 63.7}
{'sin': 789012345, 'name': 'Matthew', 'lot': 74.6, 'age': 26, 'department_name': 'Sales', 'budget': 63.7}


List all students who are enrolled in the Mathematics course.

In [20]:
# Perform the join between Students and Courses collections
join_result = db.Students.aggregate([
    {
        "$lookup": {
            "from": "Courses",  # The second collection (Courses)
            "localField": "StuID",  # Field in Students collection
            "foreignField": "SID",  # Field in Courses collection
            "as": "courses_info"  # Output field to store matching courses data
        }
    },
    {
        "$unwind": "$courses_info"  # Flatten the array of courses (if there are multiple matches)
    },
    {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "StuID": 1,
            "Name": 1,
            "Age": 1,
            "Status": 1,
            "CourseName": "$courses_info.CourseName",  # Include course name from Courses collection
            "CourseID": "$courses_info.CourseID"  # Include course ID from Courses collection
        }
    }
])

# Print the join result
for document in join_result:
    print(document)


{'StuID': 1, 'Name': 'Alice', 'Age': 20, 'Status': 'Intl.', 'CourseName': 'Mathematics', 'CourseID': 101}
{'StuID': 2, 'Name': 'Bob', 'Age': 22, 'Status': 'Dmstc.', 'CourseName': 'Physics', 'CourseID': 102}
{'StuID': 3, 'Name': 'Charlie', 'Age': 21, 'Status': 'Dmstc.', 'CourseName': 'Chemistry', 'CourseID': 103}
{'StuID': 4, 'Name': 'Dana', 'Age': 19, 'Status': 'Intl.', 'CourseName': 'Mathematics', 'CourseID': 101}


## 8 Tasks

1. Using S2 collection, select rating above 8 and project the sname and rating attributes only.
Expected answer is on page 84 in Lecture Note.

In [ ]:
# Query to select rating above 8 and project only 'sname' and 'rating' fields
result = db.S2.aggregate([
    {
        "$match": { "rating": { "$gt": 8 } }  # Filter for ratings above 8
    },
    {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "sname": 1,
            "rating": 1
        }
    }
])

# Print the query result
for document in result:
    print(document)


{'sname': 'yuppy', 'rating': 9}
{'sname': 'Rusty', 'rating': 10}


2. Using S3 collection, find the age of the youngest sailor who is eligible to vote (i.e., is at least 18
years old) for each rating level with at least two such sailors. Expected answer is on page 114 in
Lecture Note - More Exercise #1.

In [ ]:
# Query to find the youngest eligible sailor (age >= 18) for each rating with at least 2 sailors
result = db.S3.aggregate([
    {
        "$match": { "age": { "$gte": 18 } }  # Filter for sailors eligible to vote (age >= 18)
    },
    {
        "$group": {
            "_id": "$rating",
            "min_age": { "$min": "$age" },  # Find the minimum age per rating
            "count": { "$sum": 1 }  # Count the number of sailors per rating
        }
    },
    {
        "$match": { "count": { "$gte": 2 } }  # Keep only ratings with at least 2 sailors
    },
    {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "rating": "$_id",
            "min_age": 1
        }
    },
    {
        "$sort": { 'rating': 1 }  # Sort by rating in ascending order
    }
])

# Print the query result
for document in result:
    print(document)


{'min_age': 25.5, 'rating': 3}
{'min_age': 35.0, 'rating': 7}
{'min_age': 25.5, 'rating': 8}


3. Using S3 collection, find the average age of sailors for each rating level that has at least two
sailors. Expected answer is on page 115 in Lecture Note - More Exercise #2

In [ ]:
# Query to find the average age of sailors for each rating with at least 2 sailors
result = db.S3.aggregate([
    {
        "$group": {
            "_id": "$rating",
            "average_age": { "$avg": "$age" },  # Calculate average age per rating
            "count": { "$sum": 1 }  # Count sailors per rating
        }
    },
    {
        "$match": { "count": { "$gte": 2 } }  # Keep only ratings with at least 2 sailors
    },
    {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "rating": "$_id",
            "average_age": 1
        }
    },
    {
        "$sort": { 'rating': 1 }  # Sort by rating in ascending order
    }
])

# Print the query result
for document in result:
    print(document)


{'average_age': 44.5, 'rating': 3}
{'average_age': 40.0, 'rating': 7}
{'average_age': 40.5, 'rating': 8}
{'average_age': 25.5, 'rating': 10}


4. Using S3 collection, find the average age of sailors who are of voting age (i.e., at least 18 years
old) for each rating level that has at least 2 sailors. Expected answer is on page 117 in Lecture
Note - More Exercise #3.

In [ ]:
# Query to find the average age of sailors for each rating with at least 2 sailors
result = db.S3.aggregate([
    {
        "$match": { "age": { "$gte": 18 } }  # Filter for sailors eligible to vote (age >= 18)
    },
    {
        "$group": {
            "_id": "$rating",
            "average_age": { "$avg": "$age" },  # Calculate average age per rating
            "count": { "$sum": 1 }  # Count sailors per rating
        }
    },
    {
        "$match": { "count": { "$gte": 2 } }  # Keep only ratings with at least 2 sailors
    },
    {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "rating": "$_id",
            "average_age": 1
        }
    },
    {
        "$sort": { 'rating': 1 }  # Sort by rating in ascending order
    }
])

# Print the query result
for document in result:
    print(document)


{'average_age': 44.5, 'rating': 3}
{'average_age': 40.0, 'rating': 7}
{'average_age': 40.5, 'rating': 8}
